# Running External Strategy Files

This notebook demonstrates how to run trading strategies defined in external Python files.

## Why Use External Strategy Files?

- ✅ **Reusability**: Share strategies across notebooks and scripts
- ✅ **Version Control**: Track strategy changes with git
- ✅ **Testing**: Easy to unit test standalone files
- ✅ **CLI Compatible**: Can run with `zipline run` command
- ✅ **Organization**: Keep notebook clean, strategy in separate file

## Strategy: SMA Crossover

We'll use the **Simple Moving Average Crossover** strategy from `strategies_files/sma_crossover.py`:

**Strategy Logic:**
- **BUY** when 50-day SMA crosses above 200-day SMA (Golden Cross)
- **SELL** when 50-day SMA crosses below 200-day SMA (Death Cross)
- Equal weight across all positions (max 10 stocks)

**📁 Strategy Location:** Click on the `strategies_files` folder in Jupyter to view/edit the strategy!

## Prerequisites

Make sure you have a bundle ingested:

```bash
# Sharadar
python scripts/manage_data.py setup --source sharadar --tickers AAPL,MSFT,GOOGL,AMZN,TSLA,META,NVDA,NFLX,AMD,INTC

# OR Yahoo Finance
python scripts/manage_data.py setup --source yahoo
```

## Step 1: Import Libraries

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pathlib import Path

# Zipline imports
from zipline import run_algorithm

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ Libraries imported successfully!")
print(f"  Current dir: {Path.cwd()}")
print(f"  Strategy files location: strategies_files/ folder (accessible in Jupyter)")

## Step 2: Import Strategy Functions

Import the strategy from the external file:

In [ ]:
# Import strategy functions from the strategies_files folder
# This folder is accessible in the notebooks directory
from strategies_files.sma_crossover import (
    initialize,
    before_trading_start,
    rebalance,
    analyze,
    FAST_MA_PERIOD,
    SLOW_MA_PERIOD,
    STOCK_UNIVERSE,
)

print("✓ Strategy imported successfully!\n")
print("Strategy Parameters:")
print(f"  Fast MA: {FAST_MA_PERIOD} days")
print(f"  Slow MA: {SLOW_MA_PERIOD} days")
print(f"  Universe: {len(STOCK_UNIVERSE)} stocks")
print(f"  Stocks: {', '.join(STOCK_UNIVERSE)}")
print(f"\n💡 Tip: You can edit this strategy by clicking on 'strategies_files' folder in Jupyter")

## Step 3: Check Bundle Status (Important!)

**⚠️ You must ingest bundle data before running a backtest!**

Run this cell to check if you have data:

In [ ]:
# Run this cell to check what data you have available
%run check_data_range.py

# Or specify a specific bundle:
# %run check_data_range.py sharadar
# %run check_data_range.py yahoo

In [ ]:
## Step 4: Configure Backtest Parameters

**Important:** Make sure your dates match the data you ingested!

In [ ]:
# Backtest configuration
# IMPORTANT: Dates must match the data you have ingested!
# Use timezone-naive dates for Zipline

# Example date ranges:
# Yahoo Finance: Typically has data from ~2010 to present
# Sharadar: Depends on your subscription and what you ingested

START_DATE = pd.Timestamp('2020-01-02')  # Adjust to your data range
END_DATE = pd.Timestamp('2021-12-31')    # Adjust to your data range
CAPITAL_BASE = 100000  # $100,000 starting capital
BUNDLE_NAME = 'yahoo'  # Change to 'sharadar' if you ingested Sharadar data

print("Backtest Configuration:")
print(f"  Start Date: {START_DATE.date()}")
print(f"  End Date: {END_DATE.date()}")
print(f"  Capital: ${CAPITAL_BASE:,}")
print(f"  Bundle: {BUNDLE_NAME}")
print(f"\n💡 Note: Dates are timezone-naive (required by exchange_calendars)")
print(f"⚠️  Make sure these dates match the data you ingested in Step 3!")

## Step 5: Run Backtest

**After verifying your bundle status and date range, run the backtest:**

In [ ]:
print("\nRunning backtest...\n")
print("="*60)

# Run the strategy
results = run_algorithm(
    start=START_DATE,
    end=END_DATE,
    initialize=initialize,
    before_trading_start=before_trading_start,
    analyze=analyze,
    capital_base=CAPITAL_BASE,
    bundle=BUNDLE_NAME,
)

print("="*60)
print("\n✓ Backtest complete!")

## Step 6: Analyze Results

In [ ]:
# Calculate key metrics
returns = results['returns']
cumulative_returns = (1 + returns).cumprod() - 1

print("\n=== PERFORMANCE METRICS ===\n")

# Total return
total_return = (results['portfolio_value'].iloc[-1] / results['portfolio_value'].iloc[0] - 1) * 100
print(f"Total Return: {total_return:+.2f}%")

# Annualized return
days = (END_DATE - START_DATE).days
years = days / 365.25
annualized_return = ((1 + total_return/100) ** (1/years) - 1) * 100
print(f"Annualized Return: {annualized_return:+.2f}%")

# Volatility
volatility = returns.std() * np.sqrt(252) * 100
print(f"Volatility (annualized): {volatility:.2f}%")

# Sharpe ratio
sharpe = returns.mean() / returns.std() * np.sqrt(252)
print(f"Sharpe Ratio: {sharpe:.2f}")

# Maximum drawdown
cumulative = (1 + returns).cumprod()
running_max = cumulative.cummax()
drawdown = (cumulative / running_max) - 1
max_drawdown = drawdown.min() * 100
print(f"Maximum Drawdown: {max_drawdown:.2f}%")

# Win rate
winning_days = (returns > 0).sum()
total_days = len(returns)
win_rate = (winning_days / total_days) * 100
print(f"Win Rate: {win_rate:.1f}%")

# Trading stats
print(f"\nTrading Statistics:")
print(f"  Average Positions: {results['num_positions'].mean():.1f}")
print(f"  Max Positions: {results['num_positions'].max()}")
print(f"  Total Trades: {results['trades_made'].iloc[-1]}")

# Final values
print(f"\nFinal Portfolio:")
print(f"  Portfolio Value: ${results['portfolio_value'].iloc[-1]:,.2f}")
print(f"  Cash: ${results['cash'].iloc[-1]:,.2f}")
print(f"  Positions Value: ${results['portfolio_value'].iloc[-1] - results['cash'].iloc[-1]:,.2f}")

## Step 7: Visualize Performance

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12))
fig.suptitle('SMA Crossover Strategy Performance', fontsize=16, fontweight='bold')

# Portfolio value
ax1 = axes[0]
ax1.plot(results.index, results['portfolio_value'], linewidth=2, label='Portfolio Value')
ax1.axhline(y=CAPITAL_BASE, color='red', linestyle='--', alpha=0.5, label='Initial Capital')
ax1.set_title('Portfolio Value Over Time')
ax1.set_ylabel('Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Cumulative returns
ax2 = axes[1]
ax2.plot(cumulative_returns.index, cumulative_returns * 100, linewidth=2, color='green')
ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax2.set_title('Cumulative Returns')
ax2.set_ylabel('Return (%)')
ax2.grid(True, alpha=0.3)

# Drawdown
ax3 = axes[2]
ax3.fill_between(drawdown.index, drawdown * 100, 0, alpha=0.3, color='red')
ax3.plot(drawdown.index, drawdown * 100, linewidth=1, color='darkred')
ax3.set_title('Drawdown')
ax3.set_xlabel('Date')
ax3.set_ylabel('Drawdown (%)')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Trading activity
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
fig.suptitle('Trading Activity', fontsize=16, fontweight='bold')

# Number of positions over time
ax1.plot(results.index, results['num_positions'], linewidth=2, color='blue')
ax1.set_title('Number of Positions Over Time')
ax1.set_ylabel('Number of Stocks')
ax1.grid(True, alpha=0.3)

# Cash allocation
cash_pct = (results['cash'] / results['portfolio_value']) * 100
ax2.plot(cash_pct.index, cash_pct, linewidth=2, color='orange')
ax2.set_title('Cash Allocation')
ax2.set_xlabel('Date')
ax2.set_ylabel('Cash (%)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 8: Monthly Returns Analysis

In [ ]:
# Calculate monthly returns
monthly_returns = returns.resample('M').apply(lambda x: (1 + x).prod() - 1)

print("\n=== MONTHLY RETURNS ===\n")
print(monthly_returns.to_frame('Return').applymap(lambda x: f"{x*100:+.2f}%"))

# Plot monthly returns
fig, ax = plt.subplots(figsize=(14, 6))
colors = ['green' if x > 0 else 'red' for x in monthly_returns]
monthly_returns.plot(kind='bar', ax=ax, color=colors, alpha=0.7)
ax.set_title('Monthly Returns', fontsize=14, fontweight='bold')
ax.set_xlabel('Month')
ax.set_ylabel('Return')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.grid(True, alpha=0.3, axis='y')

# Format y-axis as percentage
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y*100:.0f}%'))

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Monthly stats
print(f"\nMonthly Statistics:")
print(f"  Average Monthly Return: {monthly_returns.mean()*100:+.2f}%")
print(f"  Best Month: {monthly_returns.max()*100:+.2f}%")
print(f"  Worst Month: {monthly_returns.min()*100:+.2f}%")
print(f"  Positive Months: {(monthly_returns > 0).sum()}/{len(monthly_returns)}")

## Step 9: Export Results

In [ ]:
# Export results to CSV
output_dir = '/data/backtest_results'
import os
os.makedirs(output_dir, exist_ok=True)

# Export full results
output_file = f"{output_dir}/sma_crossover_results_{END_DATE.date()}.csv"
results.to_csv(output_file)
print(f"✓ Results exported to: {output_file}")

# Export summary
summary = pd.DataFrame({
    'Metric': [
        'Total Return (%)',
        'Annualized Return (%)',
        'Volatility (%)',
        'Sharpe Ratio',
        'Max Drawdown (%)',
        'Win Rate (%)',
        'Total Trades',
        'Final Portfolio Value',
    ],
    'Value': [
        f"{total_return:.2f}",
        f"{annualized_return:.2f}",
        f"{volatility:.2f}",
        f"{sharpe:.2f}",
        f"{max_drawdown:.2f}",
        f"{win_rate:.1f}",
        f"{results['trades_made'].iloc[-1]:.0f}",
        f"${results['portfolio_value'].iloc[-1]:,.2f}",
    ]
})

summary_file = f"{output_dir}/sma_crossover_summary_{END_DATE.date()}.csv"
summary.to_csv(summary_file, index=False)
print(f"✓ Summary exported to: {summary_file}")

## Alternative: Run from Command Line

You can also run this strategy directly from the command line:

```bash
# Inside Docker container - using the strategies_files copy
zipline run \
    -f /home/user/zipline-reloaded/notebooks/strategies_files/sma_crossover.py \
    -b sharadar \
    --start 2022-01-01 \
    --end 2023-12-31 \
    --capital-base 100000 \
    -o /data/backtest_results/sma_results.pickle

# Or using the main strategies directory
zipline run \
    -f /home/user/zipline-reloaded/strategies/sma_crossover.py \
    -b sharadar \
    --start 2022-01-01 \
    --end 2023-12-31 \
    --capital-base 100000 \
    -o /data/backtest_results/sma_results.pickle
```

## Customizing the Strategy

To modify the strategy:

1. **Easy Way:** Click on `strategies_files` folder in Jupyter → Open `sma_crossover.py` → Edit directly
2. Change parameters:
   - `FAST_MA_PERIOD`: Fast moving average period
   - `SLOW_MA_PERIOD`: Slow moving average period
   - `STOCK_UNIVERSE`: List of stocks to trade
   - `MAX_POSITIONS`: Maximum number of positions
3. Save the file
4. Restart this notebook kernel (Kernel → Restart)
5. Re-run all cells

## Creating Your Own Strategy

To create a new strategy file:

1. In Jupyter, navigate to the `strategies_files` folder
2. Click **New** → **Text File**
3. Copy the contents of `sma_crossover.py` as a template
4. Save as `my_strategy.py`
5. Edit your strategy logic
6. Import it in a notebook: `from strategies_files.my_strategy import ...`

## Try the EMA Strategy Too!

We also have a faster EMA crossover strategy:

```python
# Change the import at the top to:
from strategies_files.ema_crossover import (
    initialize, before_trading_start, rebalance, analyze,
    FAST_EMA_PERIOD, SLOW_EMA_PERIOD, STOCK_UNIVERSE,
)
```

The EMA strategy uses 10/20 day periods for more frequent trading signals.

## Resources

- [Zipline API Reference](https://zipline.ml4trading.io/api-reference.html)
- [Strategy Files](../strategies_files/) - Click to view/edit
- [Research Notebook](./07_pipeline_research.ipynb)
- [Quick Links](./00_quick_links.ipynb) - Helper for accessing files